# Case: Cloud Engineer / Developer in Cloud Analytics og MLOps

In [1]:
"""
@author: hosseind
"""

'\n@author: hosseind\n'

Developing a solution in Python for a requested interview case which exposes an API with given specifications.

### Loading reqired libraries

In [2]:
from fastapi import FastAPI, Query
import sqlite3
import uvicorn
import json
import os

### Define the path to the IMDb SQLite database file

Connect to the sqlite database using the absolute path to avoid bugs:

In [3]:
DATABASE_PATH = os.path.abspath(os.path.join(os.path.dirname('/Users/hosseind/Downloads/'),'movie.sqlite'))

### Run SQL-requests against the database to get tables info

In [4]:
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()
cursor.execute("SELECT count(*) FROM sqlite_master WHERE type='table';")
table_count = cursor.fetchone()[0]
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f'Total tables in the database: {table_count}')
for table in tables:
    cursor.execute(f"PRAGMA table_info({table[0]})")
    column_info = cursor.fetchall()
    column_names = [info[1] for info in column_info]
    print(table[0])
    print(column_names)
    
#df = pd.read_sql_query(f"SELECT * from {table[0]}", conn)
#print(df.head())
conn.close()

Total tables in the database: 3
movies
['id', 'original_title', 'budget', 'popularity', 'release_date', 'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline', 'uid', 'director_id']
sqlite_sequence
['name', 'seq']
directors
['name', 'id', 'gender', 'uid', 'department']


## Create the interface to run SQL-requests against the database

In [23]:
# define a function to execute a given SQL query
# Create a new cursor object for each request
def execute_sql_query(query):
    try:
        conn = sqlite3.connect(DATABASE_PATH)
        cursor = conn.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        return result
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return None
    
    
# SQL requests (queries)

#SQL-request that returns all directors
def get_all_directors():
    query = "SELECT DISTINCT name FROM directors GROUP BY id"
    return execute_sql_query(query)

#SQL-request that returns all films names
def get_all_films():
    query = "SELECT original_title FROM movies"
    return execute_sql_query(query)

# SQL-request that returns the five directors whose films have the highest average (vote) range
def get_top_directors_with_highest_avg_rating(limit=5):
    query = f"""
    SELECT DISTINCT name, vote_average FROM directors AS t1
    JOIN movies AS t2 ON t1.id = t2.director_id
    ORDER BY vote_average DESC
    LIMIT {limit}
    """
    return execute_sql_query(query)

### Run SQL-requests

In [24]:
print("All Directors:")
directors = get_all_directors()
i , N = 0, 5
for director in directors:
    print(director[0])
    if i == N: i = 0; break 
    else: i += 1

print("All Films:")
films = get_all_films()
for film in films:
    print(film[0])
    if i == N: break 
    else: i += 1
        
print("Top Directors with Highest Average Rating:")
top_directors = get_top_directors_with_highest_avg_rating(5)
for director in top_directors:
    print(f"{director[0]} - Average Rating: {director[1]:.2f}")

All Directors:
James Cameron
Gore Verbinski
Sam Mendes
Christopher Nolan
Andrew Stanton
Sam Raimi
All Films:
Avatar
Pirates of the Caribbean: At World's End
Spectre
The Dark Knight Rises
John Carter
Spider-Man 3
Top Directors with Highest Average Rating:
Gary Sinyor - Average Rating: 10.00
Tim McCanlies - Average Rating: 10.00
Rohit Jugraj - Average Rating: 9.50
Lance Hool - Average Rating: 9.30
Frank Darabont - Average Rating: 8.50
